<a href="https://colab.research.google.com/github/ejdogar/NLP/blob/main/Assignment_4_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd

In [37]:
dataset = pd.read_csv("/content/drive/MyDrive/Atomcamp/Assignments/Assignment-4/Pak Suzuki Car Reviews.csv")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


# Step 1: Data Preprocessing

#### 1.1 Converting Date Column to DateTime Format

In [38]:
dataset["Year"] = pd.to_datetime(dataset["Date"]).dt.year
dataset.drop("Date", axis=1, inplace = True)
dataset["Year"].astype(int)
dataset.head()

,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,0,2023 Suzuki Bolan,1,\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1,2023
1,1,2022 Suzuki Swift GLX CVT,4,"\nA good looking car with best fuel ? economy,...",5,4,5,4,5,2023
2,2,2022 Suzuki Cultus Auto Gear Shift,5,\nfor me i would must buy because its will sav...,5,5,5,5,5,2023
3,3,2007 Suzuki Liana RXi (CNG),5,\nI have used it for more than 5 years. litera...,5,5,5,5,5,2023
4,4,2022 Suzuki Alto VX,4,\nExterior: 10/10 on design but the quality us...,5,5,5,4,5,2023


#### 1.2 Creating a Model Make Year Column
Splitting the Model Column into Model Name and Year Column, in order to enhance model answering capability.

In [39]:
dataset["Make_Year"] = dataset["Model"].str.split(" ", expand=True)[0]
dataset["Model"] = dataset["Model"].str.split(" ").str[1:]


In [40]:
dataset["Model"] = dataset["Model"].str.join(" ")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year,Make_Year
0,0,Suzuki Bolan,1,\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1,2023,2023
1,1,Suzuki Swift GLX CVT,4,"\nA good looking car with best fuel ? economy,...",5,4,5,4,5,2023,2022
2,2,Suzuki Cultus Auto Gear Shift,5,\nfor me i would must buy because its will sav...,5,5,5,5,5,2023,2022
3,3,Suzuki Liana RXi (CNG),5,\nI have used it for more than 5 years. litera...,5,5,5,5,5,2023,2007
4,4,Suzuki Alto VX,4,\nExterior: 10/10 on design but the quality us...,5,5,5,4,5,2023,2022


#### 1.3 Fixing Missing Values
a. Checking indexe of car models where make year has not been mentioned.

b. Assigning the values from the last index at the very index as mostly the make year of adjacent models would be the same.

In [41]:
zero_idx = []
for index, year in dataset["Make_Year"].items():
  if len(year) == 0:
    zero_idx.append(index)
print(f"empty indexes are located at {zero_idx} indexes")
print(f"out of total {len(dataset)} car records, {len(zero_idx)} have not mentioned about car make year.")

empty indexes are located at [718, 807, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045,

In [42]:
for index in zero_idx:
  dataset["Make_Year"][index] = dataset["Make_Year"][index-1]

<ipython-input-42-8ea51966f166>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Make_Year"][index] = dataset["Make_Year"][index-1]


#### 1.4 Formatting our reviews

In [43]:
dataset["Review"] = dataset["Review"].str.replace("\n", "")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year,Make_Year
0,0,Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023,2023
1,1,Suzuki Swift GLX CVT,4,"A good looking car with best fuel ? economy,Dr...",5,4,5,4,5,2023,2022
2,2,Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023,2022
3,3,Suzuki Liana RXi (CNG),5,I have used it for more than 5 years. literall...,5,5,5,5,5,2023,2007
4,4,Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023,2022


In [44]:
import re
for index, review in enumerate(dataset['Review']):
  review = re.sub("[?.,$]", " ", review)
  dataset["Review"][index] = review
for index, review in enumerate(dataset['Make_Year']):
  review = re.sub("[?.,$/]", " ", review)
  dataset["Make_Year"][index] = review

dataset.head()


<ipython-input-44-dd1bb4997b23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Review"][index] = review
<ipython-input-44-dd1bb4997b23>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Make_Year"][index] = review


,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year,Make_Year
0,0,Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023,2023
1,1,Suzuki Swift GLX CVT,4,A good looking car with best fuel economy Dr...,5,4,5,4,5,2023,2022
2,2,Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023,2022
3,3,Suzuki Liana RXi (CNG),5,I have used it for more than 5 years literall...,5,5,5,5,5,2023,2007
4,4,Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023,2022


#### 1.5 Dropping Unnamed Column
As this column just have indexes and is of no significance as we have system indexing available for out dataset.

In [45]:
dataset.drop("Unnamed: 0", axis=1, inplace=True)
dataset.head()

,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year,Make_Year
0,Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023,2023
1,Suzuki Swift GLX CVT,4,A good looking car with best fuel economy Dr...,5,4,5,4,5,2023,2022
2,Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023,2022
3,Suzuki Liana RXi (CNG),5,I have used it for more than 5 years literall...,5,5,5,5,5,2023,2007
4,Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023,2022


1.6 Coverting Rating from Numbers to Wrod Representation

In [48]:
print(f"Unique values in Overall_rating column are: {dataset['Overall_rating'].unique()}")
print(f"Unique values in Setyle column are: {dataset['Style'].unique()}")
print(f"Unique values in Comfort column are: {dataset['Comfort'].unique()}")
print(f"Unique values in Fuel_Economy column are: {dataset['Fuel_Economy'].unique()}")
print(f"Unique values in Performance column are: {dataset['Performance'].unique()}")
print(f"Unique values in Value_for_Money column are: {dataset['Value_for_Money'].unique()}")

Unique values in Overall_rating column are: [1 4 5 3 2 0]
Unique values in Setyle column are: [1 5 4 2 3 0]
Unique values in Comfort column are: [1 4 5 3 2 0]
Unique values in Fuel_Economy column are: [1 5 2 3 4 0]
Unique values in Performance column are: [1 4 5 3 2 0]
Unique values in Value_for_Money column are: [1 5 3 4 2 0]


In [49]:
rating_dic = {0: "Worst", 1: "Bad", 2: "Substandard", 3: "Average", 4: "Good", 5: "Excellent"}
dataset[["Overall_rating", "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]] = dataset[["Overall_rating",  "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]].applymap(rating_dic.get)
dataset.head()

,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year,Make_Year
0,Suzuki Bolan,Bad,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,Bad,Bad,Bad,Bad,Bad,2023,2023
1,Suzuki Swift GLX CVT,Good,A good looking car with best fuel economy Dr...,Excellent,Good,Excellent,Good,Excellent,2023,2022
2,Suzuki Cultus Auto Gear Shift,Excellent,for me i would must buy because its will save ...,Excellent,Excellent,Excellent,Excellent,Excellent,2023,2022
3,Suzuki Liana RXi (CNG),Excellent,I have used it for more than 5 years literall...,Excellent,Excellent,Excellent,Excellent,Excellent,2023,2007
4,Suzuki Alto VX,Good,Exterior: 10/10 on design but the quality used...,Excellent,Excellent,Excellent,Good,Excellent,2023,2022


In [50]:
dataframe = dataset.astype(str)
dataframe.head()
dataframe["Make_Year"].astype(int)
dataframe["Year"].astype(int)

0       2023
1       2023
2       2023
3       2023
4       2023
        ... 
1094    2009
1095    2009
1096    2009
1097    2009
1098    2009
Name: Year, Length: 1099, dtype: int64

### Importing Necessary Libraries For Our Model

In [18]:
# !pip install accelerate==0.22.0 peft==0.5.0 bitsandbytes==0.40.2 transformers==4.32.0 langchain --quiet
# !pip install tiktoken --quiet
# !pip install sentence_transformers lark auto-gptq optimum --quiet

# Step 2: Creating Data Chunk From our CSV File

In [51]:
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(dataframe, page_content_column="Model")
documents = loader.load()
documents[0].page_content

'Suzuki Bolan'

In [52]:
documents[0]

Document(page_content='Suzuki Bolan', metadata={'Overall_rating': 'Bad', 'Review': 'is gadi k baray me 3 lafaz kahunga 1 bakwaaas 2 bakwaaaaaaas 3 bakwaaaassaaaaaaaaasfazool tareen or gharya tareen gadi shame on Suzuki company dunya me har cheez badal gai comfort level kahan se kahan ponch gya par ye farsoooda gadi he koi isko lyne ka sochy bhi na', 'Style': 'Bad', 'Comfort': 'Bad', 'Fuel_Economy': 'Bad', 'Performance': 'Bad', 'Value_for_Money': 'Bad', 'Year': '2023', 'Make_Year': '2023'})

# Step 3: Creating Embeddings of Our Chunks
Generating Embaddings based on our above created chunks.

In [53]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Step 4: Storing Embeddings in Vector Store

In [22]:
# !pip install faiss-gpu

In [54]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings,)

In [55]:
docs = db.similarity_search("Alto vxr 2006")
docs

[Document(page_content='Suzuki Alto VXR', metadata={'Overall_rating': 'Average', 'Review': 'Its an amazing purchase as it would be  daily driven car fuel economy and comfort is amazing spacious car I recommend to buy this car if you want to have economical and  comfortable ride I recommend it to my uncle now he is loving the purchase  Something which lack in this car is its road grip as its a mini hatchback so it lost road grip when exceeds limit of 110 As a reviewer we should also focus on setbacks but its setbacks are vanished out when we see its advantages I Give 8 out of 10 for comfort and fuel economy ', 'Style': 'Bad', 'Comfort': 'Average', 'Fuel_Economy': 'Excellent', 'Performance': 'Excellent', 'Value_for_Money': 'Excellent', 'Year': '2023', 'Make_Year': '2022'}),
 Document(page_content='Suzuki Alto VXR', metadata={'Overall_rating': 'Excellent', 'Review': 'Very Comfortable  And Reliable Car Comfort isAslo Very Good Fuel Economy is verybest MY Alto Gives 26 km In On Liter ANDALS

# Step 5: LLM

In [56]:
import torch
import transformers
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, GPTQConfig

In [57]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/llama-2-7b-Chat-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                        )

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [23]:
# model_id = "google/flan-t5-base"
# device = "cuda"
# dtype = torch.bfloat16
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=device)

# Step 6: Defining a Pipeline for LLM

In [59]:
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 256,
                do_sample=True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.2
                )

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [94]:
print(llm(prompt="Which car was disliked the most in year 2022?"))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




Answer: The worst-selling car of 2022 is subjective and can vary depending on various factors such as location, market trends, consumer preferences, etc. However, based on some reports and industry insights, here are a few cars that were not well received by consumers in 2022:
1. Fiat Chrysler Automobiles (FCA) - FCA's sales declined significantly in 2022 due to several reasons including poor product quality, lack of innovation, and unimpressive design.
2. Ford Motor Company - Ford faced criticism for its slow response to changing consumer preferences towards electric vehicles and autonomous driving technologies. Additionally, some of their models like the Explorer and Mustang failed to impress buyers.
3. General Motors (GM) - GM also saw a decline in sales in 2022 due to issues with their Chevrolet brand, which struggled to compete with other brands in terms of quality and value.
It's important to note that these assessments are based on general observations and may not be entirely 

# Step 7: Creating a RAG Model

In [81]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
        llm=llm, chain_type='stuff',
        retriever=db.as_retriever()
        )

In [95]:
print(rag("Which car was disliked the most in year 2022?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 The Suzuki Cultus EURO II is a terrible car and it was disliked the most in year 2022.


# Step 8: GUI Chatbot


1.   Creaating a function working as a bridge between GUI and backend.
2.   Using Gradio for creating Chatbot GUI.



In [63]:
def llm_rag_call(prompt):
  return f"LLM Response is:\n{llm(prompt = prompt)}\n ---- -------- ----\n RAG Response is: \n{rag(prompt)['result']}"

In [29]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
# !pip install gradio

In [64]:
import gradio as gr

demo = gr.Interface(fn=llm_rag_call, inputs="textbox", outputs="textbox")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://95b0251a7b50dca430.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
